# Greenhouse Berry Production Optimization

## 1. Project Setup

This notebook implements a linear programming model to determine the optimal production plan for a set of 24 greenhouses. The goal is to maximize total profit by selecting the best scenario (fruit, variety, planting type) for each greenhouse.

### Libraries

In [1]:
import pandas as pd
import pulp

### 2. Data Loading & Pre-processing

Here, we load the datasets and define key parameters for the model.

In [2]:
# Load datasets from the 'data/' directory
dproduction = pd.read_csv('data/Production.csv')
dprices = pd.read_csv('data/Prices.csv')
dsimulation = pd.read_csv('data/Simulation.csv')
dcharges = pd.read_csv('data/Charges_var.csv')

# Define global parameters
num_greenhouses = 24
num_scenarios = 15

# Map scenario indices (0-14) to their actual numbers (1, 2, 3, etc.)
# This is necessary because the data files use scenario numbers, but Python uses 0-based indices.
index_to_scenario = {
    0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 8, 6: 9, 7: 10, 8: 11,
    9: 12, 10: 13, 11: 14, 12: 15, 13: 18, 14: 19
}

# Define the weeks associated with each planting month
planting_weeks_by_month = {
    'Juin': [22, 23, 24, 25, 26],
    'Juillet': [27, 28, 29, 30, 31],
    'Aout': [31, 32, 33, 34, 35],
    'Mai': [18, 19, 20, 21, 22],
    'Avril': [14, 15, 16, 17, 18],
    'Novembre': [44, 45, 46, 47, 48],
    'Octobre': [40, 41, 42, 43, 44]
}

# Define greenhouse sectors
sectors = {
    1: [1, 2, 3, 4, 5, 6],
    2: [7, 8, 9, 10, 11],
    3: [12, 13, 14, 15, 16, 17],
    4: [18, 19],
    5: [20, 21],
    6: [22, 23, 24]
}

### 3. Model Formulation

#### 3.1 Generating Possible Combinations
We first generate a list of all valid combinations of (greenhouse, scenario, week) based on the problem's constraints.

In [3]:
possible_combinations = []

for i in range(num_greenhouses):
    for j in range(num_scenarios):
        # Get greenhouse and scenario details
        sector = dsimulation["Secteur"][i]
        surface = dsimulation["Surface"][i]
        month = dproduction["Mois"][j]
        
        # Actual scenario number from our map
        scenario_num = index_to_scenario[j]

        # Constraint: Scenarios 4 and 5 only in Sector 6
        if scenario_num in [4, 5] and sector != 6:
            continue
            
        # Constraint: Scenarios 12, 13, 14, 18 only in Sector 5 and surface < 2.87ha
        if scenario_num in [12, 13, 14, 18] and (sector != 5 or surface >= 2.87):
            continue
            
        # If constraints are met, add the combinations for each possible week
        possible_weeks = planting_weeks_by_month[month]
        for t in possible_weeks:
            possible_combinations.append((i, j, t))

print(f"Total valid combinations generated: {len(possible_combinations)}")

#### 3.2 Defining Decision Variables

In [4]:
# Create a dictionary of binary decision variables for each valid combination
X = pulp.LpVariable.dicts("X", possible_combinations, cat='Binary')

#### 3.3 Helper Functions for Objective Function
These functions calculate the revenue (CA) and variable costs (CV) for any given combination.

In [5]:
def get_production(scenario_idx, week_num):
    return dproduction[f"W{week_num}"][scenario_idx]

def get_price(scenario_idx, time_index):
    culture = dproduction["Culture"][scenario_idx]
    # Ensure the time index for price lookup is within bounds (1 to 90)
    if 1 <= time_index <= len(dprices):
        return dprices[culture][time_index - 1] # -1 for 0-based index
    return 0

def calculate_revenue(combination):
    i, j, t = combination
    surface = dsimulation["Surface"][i]
    delay = dproduction["Delai"][j]
    total_revenue = 0
    for week_of_prod in range(1, 38):
        price_week_index = t + week_of_prod + delay
        weekly_prod = get_production(j, week_of_prod)
        weekly_price = get_price(j, price_week_index)
        total_revenue += weekly_prod * surface * weekly_price
    return total_revenue

def calculate_variable_cost(combination):
    i, j, _ = combination
    surface = dsimulation["Surface"][i]
    cost_per_ha = dcharges["Cout"][j]
    return surface * cost_per_ha

def calculate_total_production(combination):
    i, j, _ = combination
    surface = dsimulation["Surface"][i]
    total_prod = 0
    for week_of_prod in range(1, 38):
        total_prod += get_production(j, week_of_prod)
    return total_prod * surface

#### 3.4 Pre-calculating Profit and Production for All Combinations
To make the model definition cleaner and faster, we pre-calculate the profit and total production for every valid combination.

In [6]:
profit_map = {combo: calculate_revenue(combo) - calculate_variable_cost(combo) for combo in possible_combinations}
production_map = {combo: calculate_total_production(combo) for combo in possible_combinations}

### 4. Defining the PuLP Model

#### 4.1 Initialize the Model

In [7]:
# Create a maximization problem
model = pulp.LpProblem("Red_Fruit_Production_Optimization", pulp.LpMaximize)

#### 4.2 Define the Objective Function

In [8]:
# The objective is to maximize the sum of profits from all chosen combinations
model += pulp.lpSum([profit_map[combo] * X[combo] for combo in possible_combinations])

#### 4.3 Define the Constraints

In [9]:
# Constraint 1: Each greenhouse must be assigned exactly one scenario and start week.
for i in range(num_greenhouses):
    model += pulp.lpSum([X[combo] for combo in possible_combinations if combo[0] == i]) == 1

# --- OPTIONAL CONSTRAINTS ---
# To activate a constraint, uncomment the relevant lines of code.

# Optional Constraint 2: Total production limit.
# threshold = 600000
# model += pulp.lpSum([production_map[combo] * X[combo] for combo in possible_combinations]) <= threshold

# Optional Constraint 3: Scenario consistency within each sector.
# for sector_id, greenhouse_list in sectors.items():
#     first_greenhouse_idx = greenhouse_list[0] - 1
#     # Find all scenarios possible for the first greenhouse in the sector
#     possible_scenarios_for_sector = list(set([c[1] for c in possible_combinations if c[0] == first_greenhouse_idx]))
#     for j in possible_scenarios_for_sector:
#         for i_gh in greenhouse_list[1:]:
#             # The decision to use scenario 'j' must be the same for all greenhouses in the sector
#             model += pulp.lpSum([X[c] for c in possible_combinations if c[0] == (i_gh - 1) and c[1] == j]) == \
#                      pulp.lpSum([X[c] for c in possible_combinations if c[0] == first_greenhouse_idx and c[1] == j])

### 5. Solve the Model and Display Results

In [10]:
# Solve the model
model.solve()

# Check the status of the solution
if pulp.LpStatus[model.status] == 'Optimal':
    print("Solver found an optimal solution.")
    print(f"Status: {pulp.LpStatus[model.status]}")
    print(f"Total Optimal Profit: € {pulp.value(model.objective):,.2f}")
    print("-"*42)
    print("Optimal Plan:")
    print("-"*42)
    
    # Create a list to hold results for sorting
    results = []
    for combo in possible_combinations:
        if X[combo].varValue == 1:
            i, j, t = combo
            results.append({
                'greenhouse': i + 1,
                'scenario': index_to_scenario[j],
                'week': t
            })
            
    # Sort results by greenhouse number and print
    for res in sorted(results, key=lambda x: x['greenhouse']):
        print(f"- Greenhouse {res['greenhouse']:2}: Use Scenario {res['scenario']:2}, Start Week {res['week']}")
else:
    print(f"Solver could not find an optimal solution. Status: {pulp.LpStatus[model.status]}")

Solver found an optimal solution.
Status: Optimal
Total Optimal Profit: € 9,433,967.50
------------------------------------------
Optimal Plan:
------------------------------------------
- Greenhouse  1: Use Scenario 15, Start Week 18
- Greenhouse  2: Use Scenario 15, Start Week 18
- Greenhouse  3: Use Scenario 15, Start Week 18
- Greenhouse  4: Use Scenario 15, Start Week 18
- Greenhouse  5: Use Scenario 15, Start Week 18
- Greenhouse  6: Use Scenario 15, Start Week 18
- Greenhouse  7: Use Scenario 15, Start Week 18
- Greenhouse  8: Use Scenario 15, Start Week 18
- Greenhouse  9: Use Scenario 15, Start Week 18
- Greenhouse 10: Use Scenario 15, Start Week 18
- Greenhouse 11: Use Scenario 15, Start Week 18
- Greenhouse 12: Use Scenario 15, Start Week 18
- Greenhouse 13: Use Scenario 15, Start Week 18
- Greenhouse 14: Use Scenario 15, Start Week 18
- Greenhouse 15: Use Scenario 15, Start Week 18
- Greenhouse 16: Use Scenario 15, Start Week 18
- Greenhouse 17: Use Scenario 15, Start Week 